<h1>Elaborazione dei datasets</h1>

<p>Import delle librerie necessarie</p>

In [26]:
import pandas as pd
import os
import glob
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import numpy as np
import re
import string

<h3>Funzione per apertura datasets</h3>

In [5]:
def main_f():
    conditioner = False
    while conditioner is not True:
        mylist = [f for f in glob.glob("*.csv")]
        for elem in mylist: print (elem)
        file_path = input('Scegli un file .csv: ')
        if os.path.exists(file_path):
            print('The file exists')
            dfname = str(file_path)
            df1 = pd.read_csv(dfname)
            pd.set_option("display.max.columns", None)
            conditioner = True
        else:
            print('The specified file does NOT exist')
    return df1

<p>Chiamata alla funzione di main con stampa dei dati</p>

In [6]:
data = main_f()
print(data)

2022-medium-stories-TOWARDsDATASCIENCE.csv
2023-medium-stories-TOWARDsDATASCIENCE.csv
chatGPT-RACCOLTADATi-12-06-23.csv
chatgpt_dal_2-11-22_a_31-03-23-UPDATED.csv
chatgpt_dal_2-11-22_a_31-03-23.csv
chatgpt_dal_2-11-22_a_31-03-23langEN-UPDATED.csv
chatgpt_dal_2-11-22_a_31-03-23langEN.csv
finalDF_elaborazioneTweets26-05.csv
Scegli un file .csv: chatgpt_dal_2-11-22_a_31-03-23langEN-UPDATED.csv
The file exists
       Unnamed: 0.1  Unnamed: 0             User                       date  \
0                 0           0  GAMAnoABURA_AGE  2023-03-30 23:59:58+00:00   
1                 1           1          jizomae  2023-03-30 23:59:56+00:00   
2                 2           2             kenn  2023-03-30 23:59:52+00:00   
3                 3           3          yamaike  2023-03-30 23:59:47+00:00   
4                 4           4    tarosuke_2525  2023-03-30 23:59:45+00:00   
...             ...         ...              ...                        ...   
16479         16479       16479      

<p>Stampo le dimensioni del dataset</p>

In [7]:
print("Ampiezza dataset: "+str(len(data))+ " rows")
print("Dimensioni rows x columns: "+ str(data.shape))

Ampiezza dataset: 16484 rows
Dimensioni rows x columns: (16484, 11)


<p>Check sui valori nulli</p>

In [ ]:
np.sum(data.isnull().any(axis=1))

<h3>Checks sulle colonne per uniformarle</h3>

In [8]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'User', 'date', 'likeCount', 'source',
       'tweetText', 'hashtags', 'lang', 'replies', 'retweetCount'],
      dtype='object')

<p>funzione drop per togliere colonne irrilevanti</p>

In [9]:
def dropCols(d):
    col = input('scegli una colonna da droppare (scrivi il nome esatto): ')
    col = str(col)
    if col in d.columns:
        print ("sicuro di voler eliminare la colonna "+ col+ "?")
        ans = input("scrivi y o n")
        ans = str(ans)
        if ans == 'y':
            print("procedo..")
            d.drop([col], axis=1)
        else:
            print("non procedo")
    return d

In [10]:
stop = False
while stop is not True:
    q = (int(input("premi 1 per droppare una colonna, 2 per non farlo:  ")))
    if q!=2:
        dropCols(data)
    else:
        stop= True
        print("fine drop delle colonne")

premi 1 per droppare una colonna, 2 per non farlo:  2
fine drop delle colonne


<h3>Funzione per fixare le date dal formato U.S al formato italiano</h3>
<p>ignoro l'orario di pubblicazione dei tweets perché non mi interessa ai fini dell'analisi</p>

In [11]:
boolean = is_datetime(data['date'])

In [12]:
def dataFixing(df_name):
    #se la data non è di tipo date time
    if boolean != True:
        #eseguo la conversione in datetime
        df_name['date'] = pd.to_datetime(df_name['date'])
    return df_name['date'].dt.strftime('%d-%m-%Y')

In [13]:
data['date'] = dataFixing(data)

<h2>Funzioni per sistemare il testo dei tweets</h2>

In [14]:
data['tweetText'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 16484 entries, 0 to 16483
Series name: tweetText
Non-Null Count  Dtype 
--------------  ----- 
16484 non-null  object
dtypes: object(1)
memory usage: 128.9+ KB


In [15]:
data['tweetText'].head()

0    ChatGPTみてると、日本人への英語教育が必修である必要性は将来的になくなる気がするな。\...
1    ChatGPT類似品と独自(？)音声合成で構築は難しいのかな。\nそこらで売ってる会話ぬいぐ...
2    これだよね。ChatGPTプラグインはAIの技術的発展が続いても物理的にできないこと、リアル...
3    ChatGPTくんに、「発言はかぎかっこ、動作は小かっこでくくる」という古のPBCの流儀を教...
4    西欧諸国はこういう感じで自制があるからいいけど、中国とか北朝鮮とかだと、chatGPT習近平...
Name: tweetText, dtype: object

In [16]:
stopwordlist_en = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [17]:
def filtroLingua(df):
    options = ['it', 'en'] 
    
    #prendo solo rows che corrispondono alle lang selezionate
    return df[df['lang'].isin(options)] 

In [18]:
data = filtroLingua(data)

In [20]:
data.head()

,Unnamed: 0.1,Unnamed: 0,User,date,likeCount,source,tweetText,hashtags,lang,replies,retweetCount
5,5,5,realDDDougherty,30-03-2023,0,NaN,@BoSnerdley @MailOnline Even ChatGPT says they...,NaN,en,0,0
6,6,6,le_debut,30-03-2023,9,NaN,letting chat GPT teach me dynamic programming,NaN,en,1,0
9,9,9,WirewalkerDon,30-03-2023,0,NaN,The greatest opportunity in #AI with #ChatGPT ...,"['AI', 'ChatGPT', 'OpenAI', 'Entrepreneurship'...",en,0,0
13,13,13,bytefeedai,30-03-2023,1,NaN,ChatGPT In Education: The Pros Cons And Unknow...,"['Education', 'EdTech']",en,0,0
19,19,19,mhdodick,30-03-2023,1,NaN,@harrymccracken You should feed the same quest...,"['ChatGPT', 'ArtificialIntelligence']",en,0,0


<p>Funzione per eliminare stopwords passando una lista di stopwords</p>

In [21]:
STOPWORDS = set(stopwordlist_en)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

In [22]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [23]:
def cleaning_URLs(df):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',df)

<p>Questa funzione consente di duplicare una colonna per poter creare elaborazioni <b>senza</b> sacrificare il testo originale</p>

In [31]:
def duplicaColonna(df, col):
    df.columns
    #q = input("scegli la colonna da duplicare: ")
    #q = str(q)
    if col in df.columns:
        newCol = col+"_duplicate"
        print("NOME DELLA NUOVA COLONNA GENERATA: "+ newCol)
        df[newCol] = df.loc[:, col]
    else:
        print("colonna non presente")
    return df

In [32]:
q = input("vuoi eliminare le stopwords? y/n: ")

if q=="y":
    print("Ecco lista colonne: ")
    print(data.columns)
    #chiede il nome della colonna
    nomeCol = str(input("inserisci il nome della colonna duplicata da cui rimuovere stopwords,punteggiatura e links: "))
    #funzione per il duplicato
    data2 = duplicaColonna(data, nomeCol)
    #funzione rimozione su colonna duplicata
    try:
        data[nomeCol]= data[nomeCol].apply(lambda text: cleaning_stopwords(text))
    except:
        print("errore nella rimozione stopwords")
    try:
        data[nomeCol]= data[nomeCol].apply(lambda x: cleaning_punctuations(x))
    except:
        print("errore nella rimozione della punteggiatura")
    try:
        data[nomeCol] = data[nomeCol].apply(lambda x: cleaning_URLs(x))
    except:
        print("errore nella rimozione dei link")

vuoi eliminare le stopwords? y/n: y
Ecco lista colonne: 
Index(['Unnamed: 0.1', 'Unnamed: 0', 'User', 'date', 'likeCount', 'source',
       'tweetText', 'hashtags', 'lang', 'replies', 'retweetCount'],
      dtype='object')
inserisci il nome della colonna duplicata da cui rimuovere stopwords,punteggiatura e links: tweetText
NOME DELLA NUOVA COLONNA GENERATA: tweetText_duplicate


In [33]:
data2['tweetText'].tail()

16476              jstinchen xPIG isn’t real I ask chatGPT
16478    ChatGPT Why still smarter machines BBC Reel ht...
16481    CastleFreak horrorfilm CHATGPT StableDiffusion...
16482                   Chat gptKKKKKKK httpstcoUZOzHe7gOC
16483    keetmuise Seeing ChatGPT identify risk widespr...
Name: tweetText, dtype: object